In [1]:

import gensim.models.keyedvectors as word2vec
import gensim.downloader as api
import numpy as np
import scipy
import itertools

In [2]:
glove_vecs = {}
word_vectors = word2vec.KeyedVectors.load_word2vec_format(
'players/GoogleNews-vectors-negative300.bin', binary=True, unicode_errors='ignore')
print('loaded word vectors')

loaded word vectors


In [3]:
with open('players/glove/glove.6B.300d.txt') as infile:
    for line in infile:
        line = line.rstrip().split(' ')
        glove_vecs[line[0]] = np.array([float(n) for n in line[1:]])

In [4]:
with open('revised.txt','w') as outfile:
    with open('players/cm_wordlist.txt') as infile:
        for line in infile:
            word = line.rstrip()
            try:
                word_vectors[word]
                glove_vecs[word]
                outfile.write(word + '\n')
            except:
                continue

In [5]:
cm_wordlist = []
with open('players/cm_wordlist.txt') as infile:
    for line in infile:
        word = line.rstrip()
        cm_wordlist.append(word)
        

In [11]:

red_words = ['centaur']


def combine(words,wordvecs):
    factor = 1.0/float(len(words))
    new_word = concatenate(words[0],wordvecs)*factor
    for word in words[1:]:
        new_word += concatenate(word,wordvecs)*factor
    return new_word

def concatenate(word,wordvecs):
    concatenated = wordvecs[0][word]
    for vec in wordvecs[1:]:
        concatenated = np.hstack((concatenated,vec[word] ))
    return concatenated

In [12]:
def filter(word):
    if word in red_words:
        return False
    for red in red_words:
        if red.find(word) != -1:
            return False
        if word.find(red) != -1:
            return False
    return True
            

all_vectors = (word_vectors,glove_vecs,)
bests = {}
for clue_num in range(1,4+1):
    best_per_dist = np.inf
    best_per = ''
    best_red_word = ''
    for red_word in list(itertools.combinations(red_words,clue_num)):
        #REPLACE THIS FOR NLTK STUFF
        combined_word = combine(red_word,all_vectors)
        best_word = ''
        best_dist = np.inf
        for word in cm_wordlist:
            if not filter(word):
                continue
            #REPLACE COSINE DISTANCE FOR NLTK STUFF
            distance = scipy.spatial.distance.cosine(combined_word,concatenate(word,all_vectors))
            if distance < best_dist:
                best_dist = distance
                best_word = word
                if best_dist < best_per_dist:
                    best_per_dist = best_dist
                    best_per = best_word
                    best_red_word = red_word
    
    bests[clue_num] = (best_red_word,best_per,best_per_dist)

In [13]:
for clue_num, clue in bests.items():
    best_red_word,combined_clue,combined_score = clue
    worst = -np.inf
    best = np.inf
    worst_word = ''
    for word in best_red_word:
        dist = scipy.spatial.distance.cosine(concatenate(word,all_vectors),concatenate(combined_clue,all_vectors))
        if dist > worst:
            worst_word = word
            worst = dist
        if dist < best:
            best = dist
            
    print(worst/best,best_red_word,worst_word,combined_clue,combined_score)

1.0 ('centaur',) centaur atlas 0.6969345487376393
nan    inf
nan    inf
nan    inf
